In [23]:
# Data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper, LinearColorMapper, ColorBar
from bokeh.palettes import Spectral5, Magma, Inferno, Plasma, Viridis, Cividis 
from bokeh.transform import factor_cmap, transform

In [25]:
# Reading data
df = pd.read_csv('C:/Users/isabe/Documents/player_weekly_data.csv')
df.head()

,Name,Team,Pos,Wk,Opp,Year,Status,TeamScore,OppScore,PassYds,...,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgs,ReceivingYds,ReceivingTD,FL
0,Aaron Jones,GB,RB,2,DET,2020,0,42,21,0,...,0,0,18,168,2,4,8,68,1,0
1,Dak Prescott,DAL,QB,2,ATL,2020,0,40,39,450,...,47,34,5,18,3,0,0,0,0,1
2,Alvin Kamara,NO,RB,2,LV,2020,0,24,34,0,...,0,0,13,79,2,9,9,95,0,0
3,Cam Newton,NE,QB,2,SEA,2020,0,30,35,397,...,44,30,11,47,2,0,0,0,0,0
4,Josh Allen,BUF,QB,2,MIA,2020,0,31,28,415,...,35,24,4,19,0,0,0,0,0,0


In [26]:
# Removing NaN 
df = df.dropna()

In [27]:
# Removing K and DST positions
remove_pos = df['Pos'] != 'K'
remove_pos = df['Pos'] != 'DST'
df = df[remove_pos]

In [28]:
df.describe()

,Wk,Year,TeamScore,OppScore,PassYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgs,ReceivingYds,ReceivingTD,FL
count,1209.0,1209.0,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000,1209.000000
mean,2.0,2020.0,26.683209,26.646816,7.248966,0.047974,0.020678,0.948718,0.628619,0.739454,3.427626,0.030604,0.627792,0.911497,7.245658,0.047974,0.025641
std,0.0,0.0,8.448895,8.498159,45.189301,0.356140,0.142364,5.862583,3.898150,3.009495,14.656156,0.215054,1.592227,2.245218,20.277341,0.235889,0.177839
min,2.0,2020.0,11.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.000000,0.000000,0.000000,0.000000,-6.000000,0.000000,0.000000
25%,2.0,2020.0,20.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.0,2020.0,30.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.0,2020.0,33.000000,33.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.0,2020.0,42.000000,42.000000,450.000000,5.000000,1.000000,61.000000,37.000000,27.000000,168.000000,3.000000,12.000000,16.000000,179.000000,3.000000,2.000000


In [30]:
# Top 5 Players with the most Passing TD (all weeks combined)
top_5_assists_overall = df.groupby('Name')[['PassingTD']].sum()
top_5_assists_overall = top_5_assists_overall.sort_values(by='PassingTD', ascending=False).head(5)
top_5_assists_overall

,PassingTD
Name,
Russell Wilson,5
Josh Allen,4
Ryan Tannehill,4
Matt Ryan,4
Gardner Minshew II,3


In [31]:
# Top 5 Players With Most Interceptions (all weeks combined)
top_5_interceptions = df.groupby('Name')[['Int']].sum()
top_5_interceptions = top_5_interceptions.sort_values(by='Int', ascending=False).head(5)
top_5_interceptions

,Int
Name,
Nicholas Morrow,1
William Jackson III,1
Pierre Desir,1
Deon Bush,1
Julian Love,1


In [32]:
# Top 5 Players With Most Fumbles (all weeks combined)
top_5_fumbles = df.groupby('Name')[['FL']].sum()
top_5_fumbles = top_5_fumbles.sort_values(by='FL', ascending=False).head(5)
top_5_fumbles

,FL
Name,
Joe Burrow,2
Ezekiel Elliott,2
Nick Mullens,2
Derek Carr,2
Dak Prescott,1


In [33]:
# Teams ith the most pass completion 
teams_completion = df.groupby('Team').Cmp.sum()
teams_completion = teams_completion.sort_values(ascending=False)
teams_completion

Team
CIN    37
DAL    34
CAR    33
MIA    31
NE     30
JAX    30
PIT    29
LV     28
KC     27
ARI    26
NO     26
PHI    26
NYG    25
HOU    25
ATL    24
BUF    24
TB     23
LAC    22
SF     22
SEA    21
NYJ    21
DET    20
LAR    20
DEN    19
IND    19
WAS    19
GB     18
CHI    18
TEN    18
BAL    18
CLE    16
MIN    11
Name: Cmp, dtype: int64

In [34]:
# AVGS by Team 
avg_teams = df.groupby('Team').mean()
avg_teams

,Wk,Year,TeamScore,OppScore,PassYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgs,ReceivingYds,ReceivingTD,FL
Team,,,,,,,,,,,,,,,,,
ARI,2.0,2020.0,30.0,15.0,7.333333,0.025641,0.000000,0.974359,0.666667,0.820513,4.102564,0.051282,0.666667,0.871795,7.333333,0.025641,0.025641
ATL,2.0,2020.0,39.0,40.0,7.184211,0.105263,0.000000,0.973684,0.631579,0.894737,2.973684,0.000000,0.631579,0.947368,7.184211,0.105263,0.000000
BAL,2.0,2020.0,33.0,16.0,5.368421,0.026316,0.026316,0.631579,0.473684,0.973684,6.052632,0.026316,0.473684,0.605263,5.368421,0.026316,0.000000
BUF,2.0,2020.0,31.0,28.0,10.921053,0.105263,0.000000,0.921053,0.631579,0.605263,2.947368,0.000000,0.631579,0.921053,10.921053,0.105263,0.026316
CAR,2.0,2020.0,17.0,31.0,9.657895,0.000000,0.026316,1.105263,0.868421,0.631579,2.289474,0.052632,0.868421,1.052632,9.657895,0.000000,0.078947
CHI,2.0,2020.0,17.0,13.0,5.000000,0.052632,0.026316,0.736842,0.473684,0.842105,3.552632,0.000000,0.447368,0.710526,4.894737,0.052632,0.026316
CIN,2.0,2020.0,30.0,35.0,8.540541,0.081081,0.027027,1.648649,1.000000,0.648649,1.837838,0.000000,1.000000,1.594595,8.540541,0.081081,0.054054
CLE,2.0,2020.0,35.0,30.0,5.918919,0.054054,0.000000,0.621622,0.432432,0.945946,5.810811,0.081081,0.432432,0.567568,5.918919,0.054054,0.000000
DAL,2.0,2020.0,40.0,39.0,11.842105,0.026316,0.000000,1.263158,0.894737,0.815789,3.289474,0.105263,0.894737,1.236842,11.842105,0.026316,0.105263


In [35]:
# The figure will be rendered inline a my Jupyter Notebook
output_notebook()

Loading BokehJS ...

In [36]:
# Plot - Top 5 Passers Overall 

top_5_assists_overall = pd.DataFrame(top_5_assists_overall)
source = ColumnDataSource(top_5_assists_overall)

names_list = source.data['Name'].tolist()
p1 = figure(
       y_range = names_list,
       title='Top 5 Passers',
       x_axis_label = 'Passing TouchDowns',
       y_axis_label = '',
       plot_height=500,
       sizing_mode="stretch_width",
       toolbar_location=None
)

p1.hbar(y='Name', 
       right='PassingTD', 
       height=0.4, 
       fill_color=factor_cmap(
              'Name',
              palette=Viridis[5],
              factors=names_list
       ),
       alpha=0.8,
       source=source
)

p1.title.text_font_size='16px'
p1.title.align='center'
p1.title.text_color='#4D5656'

show(p1)

In [37]:
# Plot - Top 5 Passers Overall 

top_5_interceptions = pd.DataFrame(top_5_interceptions)
source = ColumnDataSource(top_5_interceptions)

names_list = source.data['Name'].tolist()
p2 = figure(
       y_range = names_list,
       title='Top 5 Interceptors',
       x_axis_label = 'Interceptions',
       y_axis_label = '',
       plot_height=500,
       sizing_mode="stretch_width",
       toolbar_location=None
)

p2.hbar(y='Name', 
       right='Int', 
       height=0.4, 
       fill_color=factor_cmap(
              'Name',
              palette=Viridis[5],
              factors=names_list
       ),
       alpha=0.8,
       source=source
)

p2.title.text_font_size='16px'
p2.title.align='center'
p2.title.text_color='#4D5656'

show(p2)

In [39]:
# Plot - Top 5 Fumbles

top_5_fumbles = pd.DataFrame(top_5_fumbles)
source = ColumnDataSource(top_5_fumbles)

names_list = source.data['Name'].tolist()
p3 = figure(
       y_range = names_list,
       title='Top 5 Fumbles',
       x_axis_label = 'Fumbles',
       y_axis_label = '',
       plot_height=500,
       sizing_mode="stretch_width",
       toolbar_location=None
)

p3.hbar(y='Name', 
       right='FL', 
       height=0.4, 
       fill_color=factor_cmap(
              'Name',
              palette=Viridis[5],
              factors=names_list
       ),
       alpha=0.8,
       source=source
)

p3.title.text_font_size='16px'
p3.title.align='center'
p3.title.text_color='#4D5656'

show(p3)

In [17]:
# Creating Panel

tab1 = Panel(child=p1, title="Top 5 Passers")
tab2 = Panel(child=p2, title="Top 5  Interceptors")
tab3 = Panel(child=p3, title="Top 5 Fumbles")

show(Tabs(tabs=[tab1, tab2, tab3]))